In [10]:
import pandas as pd
import numpy as np
import text_normalizer as tn
import model_evaluation_utils as meu



np.set_printoptions(precision=2, linewidth=80)

In [12]:
dataset = pd.read_csv(r'movie_reviews.csv')

In [14]:
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

# build train and test datasets
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

# normalize datasets
norm_train_reviews = tn.normalize_corpus(train_reviews)
norm_test_reviews = tn.normalize_corpus(test_reviews)

In [15]:
tokenized_train = [tn.tokenizer.tokenize(text) for text in norm_train_reviews]
tokenized_test = [tn.tokenizer.tokenize(text) for text in norm_test_reviews]

In [16]:
from collections import Counter

# build word to index vocabulary
token_counter = Counter([token for review in tokenized_train for token in review])
vocab_map = {item[0]: index+1 for index, item in enumerate(dict(token_counter).items())}
max_index = np.max(list(vocab_map.values()))
vocab_map['PAD_INDEX'] = 0
vocab_map['NOT_FOUND_INDEX'] = max_index+1
vocab_size = len(vocab_map)
# view vocabulary size and part of the vocabulary map
print('Vocabulary Size:', vocab_size)
print('Sample slice of vocabulary map:', dict(list(vocab_map.items())[10:20]))

Vocabulary Size: 77811
Sample slice of vocabulary map: {'bore': 11, 'terribly': 12, 'predictable': 13, 'interesting': 14, 'start': 15, 'middle': 16, 'film': 17, 'little': 18, 'social': 19, 'drama': 20}


In [17]:
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# get max length of train corpus and initialize label encoder
le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0
max_len = np.max([len(review) for review in tokenized_train])

## Train reviews data corpus
# Convert tokenized text reviews to numeric vectors
train_X = [[vocab_map[token] for token in tokenized_review] for tokenized_review in tokenized_train]
train_X = sequence.pad_sequences(train_X, maxlen=max_len) # pad 
## Train prediction class labels
# Convert text sentiment labels (negative\positive) to binary encodings (0/1)
train_y = le.fit_transform(train_sentiments)

## Test reviews data corpus
# Convert tokenized text reviews to numeric vectors
test_X = [[vocab_map[token] if vocab_map.get(token) else vocab_map['NOT_FOUND_INDEX'] 
           for token in tokenized_review] 
              for tokenized_review in tokenized_test]
test_X = sequence.pad_sequences(test_X, maxlen=max_len)
## Test prediction class labels
# Convert text sentiment labels (negative\positive) to binary encodings (0/1)
test_y = le.transform(test_sentiments)

# view vector shapes
print('Max length of train review vectors:', max_len)
print('Train review vectors shape:', train_X.shape, ' Test review vectors shape:', test_X.shape)

Using TensorFlow backend.


Max length of train review vectors: 1115
Train review vectors shape: (35000, 1115)  Test review vectors shape: (15000, 1115)


In [18]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM

EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(LSTM_DIM, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1115, 128)         9959808   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1115, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 10,009,281
Trainable params: 10,009,281
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True, show_layer_names=False, 
                 rankdir='LR').create(prog='dot', format='svg'))

In [23]:
batch_size = 100
model.fit(train_X, train_y, epochs=5, batch_size=batch_size, 
          shuffle=True, validation_split=0.1, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 31500 samples, validate on 3500 samples
Epoch 1/5


20400/31500 [==================>...........] - ETA: 14:42 - loss: 0.6935 - acc: 0.46 - ETA: 11:50 - loss: 0.6927 - acc: 0.54 - ETA: 10:41 - loss: 0.6927 - acc: 0.53 - ETA: 9:59 - loss: 0.6930 - acc: 0.5225 - ETA: 9:32 - loss: 0.6933 - acc: 0.504 - ETA: 9:14 - loss: 0.6929 - acc: 0.515 - ETA: 9:02 - loss: 0.6926 - acc: 0.520 - ETA: 8:52 - loss: 0.6925 - acc: 0.525 - ETA: 8:43 - loss: 0.6923 - acc: 0.525 - ETA: 8:35 - loss: 0.6921 - acc: 0.529 - ETA: 8:29 - loss: 0.6919 - acc: 0.531 - ETA: 8:24 - loss: 0.6918 - acc: 0.538 - ETA: 8:19 - loss: 0.6915 - acc: 0.550 - ETA: 8:19 - loss: 0.6912 - acc: 0.556 - ETA: 8:16 - loss: 0.6911 - acc: 0.557 - ETA: 8:12 - loss: 0.6907 - acc: 0.565 - ETA: 8:08 - loss: 0.6904 - acc: 0.571 - ETA: 8:05 - loss: 0.6901 - acc: 0.575 - ETA: 8:03 - loss: 0.6897 - acc: 0.582 - ETA: 8:00 - loss: 0.6894 - acc: 0.585 - ETA: 7:58 - loss: 0.6890 - acc: 0.586 - ETA: 7:55 - loss: 0.6882 - acc: 0.591 - ETA: 7:53 - loss: 0.6878 - acc: 0.592 - ETA: 7:50 - loss: 0.6872 - acc: 

31500/31500 [==============================] - ETA: 3:01 - loss: 0.4418 - acc: 0.793 - ETA: 2:59 - loss: 0.4413 - acc: 0.794 - ETA: 2:58 - loss: 0.4409 - acc: 0.794 - ETA: 2:56 - loss: 0.4404 - acc: 0.794 - ETA: 2:54 - loss: 0.4401 - acc: 0.795 - ETA: 2:53 - loss: 0.4401 - acc: 0.795 - ETA: 2:51 - loss: 0.4396 - acc: 0.795 - ETA: 2:49 - loss: 0.4391 - acc: 0.795 - ETA: 2:48 - loss: 0.4388 - acc: 0.795 - ETA: 2:46 - loss: 0.4383 - acc: 0.796 - ETA: 2:45 - loss: 0.4379 - acc: 0.796 - ETA: 2:43 - loss: 0.4380 - acc: 0.796 - ETA: 2:41 - loss: 0.4378 - acc: 0.796 - ETA: 2:40 - loss: 0.4374 - acc: 0.797 - ETA: 2:38 - loss: 0.4369 - acc: 0.797 - ETA: 2:36 - loss: 0.4364 - acc: 0.797 - ETA: 2:35 - loss: 0.4367 - acc: 0.797 - ETA: 2:33 - loss: 0.4368 - acc: 0.797 - ETA: 2:31 - loss: 0.4363 - acc: 0.797 - ETA: 2:30 - loss: 0.4358 - acc: 0.797 - ETA: 2:28 - loss: 0.4354 - acc: 0.798 - ETA: 2:26 - loss: 0.4347 - acc: 0.798 - ETA: 2:25 - loss: 0.4342 - acc: 0.798 - ETA: 2:23 - loss: 0.4339 - acc: 0

20400/31500 [==================>...........] - ETA: 8:48 - loss: 0.2439 - acc: 0.920 - ETA: 8:40 - loss: 0.2504 - acc: 0.925 - ETA: 8:39 - loss: 0.2624 - acc: 0.903 - ETA: 8:38 - loss: 0.2611 - acc: 0.907 - ETA: 8:37 - loss: 0.2596 - acc: 0.904 - ETA: 8:38 - loss: 0.2375 - acc: 0.915 - ETA: 8:40 - loss: 0.2277 - acc: 0.918 - ETA: 8:39 - loss: 0.2273 - acc: 0.918 - ETA: 8:36 - loss: 0.2249 - acc: 0.918 - ETA: 8:35 - loss: 0.2320 - acc: 0.916 - ETA: 8:33 - loss: 0.2379 - acc: 0.914 - ETA: 8:31 - loss: 0.2375 - acc: 0.912 - ETA: 8:30 - loss: 0.2349 - acc: 0.914 - ETA: 8:28 - loss: 0.2285 - acc: 0.917 - ETA: 8:27 - loss: 0.2262 - acc: 0.916 - ETA: 8:25 - loss: 0.2238 - acc: 0.917 - ETA: 8:23 - loss: 0.2254 - acc: 0.917 - ETA: 8:21 - loss: 0.2276 - acc: 0.915 - ETA: 8:19 - loss: 0.2291 - acc: 0.915 - ETA: 8:17 - loss: 0.2254 - acc: 0.917 - ETA: 8:15 - loss: 0.2271 - acc: 0.916 - ETA: 8:13 - loss: 0.2296 - acc: 0.915 - ETA: 8:12 - loss: 0.2330 - acc: 0.915 - ETA: 8:11 - loss: 0.2405 - acc: 0

31500/31500 [==============================] - ETA: 3:06 - loss: 0.2313 - acc: 0.913 - ETA: 3:05 - loss: 0.2315 - acc: 0.913 - ETA: 3:03 - loss: 0.2314 - acc: 0.913 - ETA: 3:01 - loss: 0.2311 - acc: 0.913 - ETA: 2:59 - loss: 0.2309 - acc: 0.913 - ETA: 2:58 - loss: 0.2315 - acc: 0.913 - ETA: 2:56 - loss: 0.2318 - acc: 0.912 - ETA: 2:54 - loss: 0.2322 - acc: 0.912 - ETA: 2:53 - loss: 0.2324 - acc: 0.912 - ETA: 2:51 - loss: 0.2324 - acc: 0.912 - ETA: 2:49 - loss: 0.2326 - acc: 0.912 - ETA: 2:47 - loss: 0.2327 - acc: 0.912 - ETA: 2:46 - loss: 0.2326 - acc: 0.912 - ETA: 2:44 - loss: 0.2324 - acc: 0.912 - ETA: 2:42 - loss: 0.2326 - acc: 0.912 - ETA: 2:41 - loss: 0.2326 - acc: 0.913 - ETA: 2:39 - loss: 0.2324 - acc: 0.913 - ETA: 2:37 - loss: 0.2325 - acc: 0.913 - ETA: 2:36 - loss: 0.2326 - acc: 0.913 - ETA: 2:34 - loss: 0.2324 - acc: 0.913 - ETA: 2:32 - loss: 0.2322 - acc: 0.913 - ETA: 2:31 - loss: 0.2321 - acc: 0.913 - ETA: 2:29 - loss: 0.2317 - acc: 0.913 - ETA: 2:27 - loss: 0.2320 - acc: 0

...

31500/31500 [==============================] - ETA: 3:09 - loss: 0.1521 - acc: 0.946 - ETA: 3:08 - loss: 0.1521 - acc: 0.946 - ETA: 3:06 - loss: 0.1527 - acc: 0.945 - ETA: 3:04 - loss: 0.1528 - acc: 0.945 - ETA: 3:02 - loss: 0.1528 - acc: 0.945 - ETA: 3:01 - loss: 0.1530 - acc: 0.945 - ETA: 2:59 - loss: 0.1529 - acc: 0.945 - ETA: 2:57 - loss: 0.1526 - acc: 0.945 - ETA: 2:56 - loss: 0.1525 - acc: 0.945 - ETA: 2:54 - loss: 0.1527 - acc: 0.945 - ETA: 2:52 - loss: 0.1529 - acc: 0.945 - ETA: 2:50 - loss: 0.1528 - acc: 0.945 - ETA: 2:49 - loss: 0.1526 - acc: 0.945 - ETA: 2:47 - loss: 0.1531 - acc: 0.945 - ETA: 2:45 - loss: 0.1532 - acc: 0.945 - ETA: 2:43 - loss: 0.1531 - acc: 0.945 - ETA: 2:42 - loss: 0.1535 - acc: 0.945 - ETA: 2:40 - loss: 0.1535 - acc: 0.945 - ETA: 2:38 - loss: 0.1538 - acc: 0.945 - ETA: 2:37 - loss: 0.1535 - acc: 0.945 - ETA: 2:35 - loss: 0.1535 - acc: 0.945 - ETA: 2:33 - loss: 0.1538 - acc: 0.945 - ETA: 2:31 - loss: 0.1537 - acc: 0.945 - ETA: 2:30 - loss: 0.1534 - acc: 0

20400/31500 [==================>...........] - ETA: 8:54 - loss: 0.1027 - acc: 0.970 - ETA: 9:06 - loss: 0.0863 - acc: 0.980 - ETA: 9:01 - loss: 0.0776 - acc: 0.983 - ETA: 8:58 - loss: 0.0890 - acc: 0.972 - ETA: 8:57 - loss: 0.0843 - acc: 0.974 - ETA: 8:55 - loss: 0.0858 - acc: 0.973 - ETA: 8:51 - loss: 0.0884 - acc: 0.970 - ETA: 8:50 - loss: 0.0831 - acc: 0.972 - ETA: 8:49 - loss: 0.0859 - acc: 0.971 - ETA: 8:46 - loss: 0.0896 - acc: 0.969 - ETA: 8:46 - loss: 0.0896 - acc: 0.970 - ETA: 8:43 - loss: 0.0869 - acc: 0.971 - ETA: 8:39 - loss: 0.0870 - acc: 0.970 - ETA: 8:37 - loss: 0.0858 - acc: 0.971 - ETA: 8:36 - loss: 0.0883 - acc: 0.970 - ETA: 8:34 - loss: 0.0874 - acc: 0.971 - ETA: 8:33 - loss: 0.0870 - acc: 0.971 - ETA: 8:32 - loss: 0.0888 - acc: 0.970 - ETA: 8:30 - loss: 0.0903 - acc: 0.970 - ETA: 8:28 - loss: 0.0900 - acc: 0.971 - ETA: 8:26 - loss: 0.0889 - acc: 0.971 - ETA: 8:24 - loss: 0.0884 - acc: 0.972 - ETA: 8:22 - loss: 0.0880 - acc: 0.972 - ETA: 8:20 - loss: 0.0881 - acc: 0

31500/31500 [==============================] - ETA: 3:08 - loss: 0.0977 - acc: 0.966 - ETA: 3:06 - loss: 0.0981 - acc: 0.966 - ETA: 3:05 - loss: 0.0982 - acc: 0.966 - ETA: 3:03 - loss: 0.0981 - acc: 0.966 - ETA: 3:01 - loss: 0.0985 - acc: 0.966 - ETA: 3:00 - loss: 0.0987 - acc: 0.965 - ETA: 2:58 - loss: 0.0990 - acc: 0.965 - ETA: 2:56 - loss: 0.0993 - acc: 0.965 - ETA: 2:54 - loss: 0.0994 - acc: 0.965 - ETA: 2:53 - loss: 0.0995 - acc: 0.965 - ETA: 2:51 - loss: 0.0995 - acc: 0.965 - ETA: 2:49 - loss: 0.0997 - acc: 0.965 - ETA: 2:47 - loss: 0.0995 - acc: 0.965 - ETA: 2:46 - loss: 0.0997 - acc: 0.965 - ETA: 2:44 - loss: 0.0997 - acc: 0.965 - ETA: 2:42 - loss: 0.1000 - acc: 0.965 - ETA: 2:41 - loss: 0.1000 - acc: 0.965 - ETA: 2:39 - loss: 0.1001 - acc: 0.965 - ETA: 2:37 - loss: 0.1007 - acc: 0.965 - ETA: 2:35 - loss: 0.1007 - acc: 0.965 - ETA: 2:34 - loss: 0.1005 - acc: 0.965 - ETA: 2:32 - loss: 0.1007 - acc: 0.965 - ETA: 2:30 - loss: 0.1010 - acc: 0.964 - ETA: 2:29 - loss: 0.1010 - acc: 0

20400/31500 [==================>...........] - ETA: 9:07 - loss: 0.0560 - acc: 0.970 - ETA: 9:08 - loss: 0.0451 - acc: 0.985 - ETA: 9:01 - loss: 0.0398 - acc: 0.986 - ETA: 9:01 - loss: 0.0457 - acc: 0.987 - ETA: 9:01 - loss: 0.0415 - acc: 0.988 - ETA: 8:59 - loss: 0.0471 - acc: 0.985 - ETA: 8:58 - loss: 0.0503 - acc: 0.982 - ETA: 8:55 - loss: 0.0535 - acc: 0.981 - ETA: 8:52 - loss: 0.0610 - acc: 0.976 - ETA: 8:49 - loss: 0.0603 - acc: 0.977 - ETA: 8:48 - loss: 0.0615 - acc: 0.975 - ETA: 8:48 - loss: 0.0598 - acc: 0.977 - ETA: 8:46 - loss: 0.0578 - acc: 0.978 - ETA: 8:44 - loss: 0.0577 - acc: 0.978 - ETA: 8:41 - loss: 0.0554 - acc: 0.980 - ETA: 8:39 - loss: 0.0552 - acc: 0.979 - ETA: 8:38 - loss: 0.0543 - acc: 0.980 - ETA: 8:36 - loss: 0.0577 - acc: 0.979 - ETA: 8:34 - loss: 0.0560 - acc: 0.980 - ETA: 8:31 - loss: 0.0625 - acc: 0.979 - ETA: 8:30 - loss: 0.0621 - acc: 0.979 - ETA: 8:28 - loss: 0.0606 - acc: 0.979 - ETA: 8:26 - loss: 0.0598 - acc: 0.980 - ETA: 8:24 - loss: 0.0581 - acc: 0

31500/31500 [==============================] - ETA: 3:09 - loss: 0.0740 - acc: 0.976 - ETA: 3:07 - loss: 0.0739 - acc: 0.976 - ETA: 3:05 - loss: 0.0737 - acc: 0.976 - ETA: 3:04 - loss: 0.0737 - acc: 0.976 - ETA: 3:02 - loss: 0.0739 - acc: 0.976 - ETA: 3:00 - loss: 0.0738 - acc: 0.976 - ETA: 2:58 - loss: 0.0739 - acc: 0.976 - ETA: 2:57 - loss: 0.0738 - acc: 0.976 - ETA: 2:55 - loss: 0.0737 - acc: 0.976 - ETA: 2:53 - loss: 0.0738 - acc: 0.976 - ETA: 2:52 - loss: 0.0747 - acc: 0.975 - ETA: 2:50 - loss: 0.0747 - acc: 0.975 - ETA: 2:48 - loss: 0.0745 - acc: 0.975 - ETA: 2:46 - loss: 0.0744 - acc: 0.975 - ETA: 2:45 - loss: 0.0744 - acc: 0.975 - ETA: 2:43 - loss: 0.0743 - acc: 0.975 - ETA: 2:41 - loss: 0.0741 - acc: 0.975 - ETA: 2:40 - loss: 0.0742 - acc: 0.975 - ETA: 2:38 - loss: 0.0739 - acc: 0.975 - ETA: 2:36 - loss: 0.0739 - acc: 0.975 - ETA: 2:34 - loss: 0.0740 - acc: 0.975 - ETA: 2:33 - loss: 0.0738 - acc: 0.975 - ETA: 2:31 - loss: 0.0740 - acc: 0.975 - ETA: 2:29 - loss: 0.0741 - acc: 0

In [24]:
pred_test = model.predict_classes(test_X)
predictions = le.inverse_transform(pred_test.flatten())